# coroutine

In [ ]:
def simple_coroutine1():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received', x)
    
my_coro1 = simple_coroutine1()
print(my_coro1)
next(my_coro1)

try:
    my_coro1.send(123)
except StopIteration as e:
    print('StopIteration')

- GEN_CREATED
- GEN_RUNNING
- GEN_SUSPENDED
- GEN_CLOSED

In [ ]:
def simple_coroutine2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Started: c =', c)
    
my_coro2 = simple_coroutine2(2)

from inspect import getgeneratorstate
print(getgeneratorstate(my_coro2))  # GEN_CREATED

next(my_coro2)
print(getgeneratorstate(my_coro2))  # GEN_SUSPENDED

try:
    my_coro2.send(13)
    my_coro2.send(125)
except StopIteration as e:
    print('StopIteration')
else:
    print('No StopIteration')
finally:
    print(getgeneratorstate(my_coro2))  # GEN_SUSPENDED

## without decorator

In [ ]:
def averager0():
    total = 0.0
    count = 0
    average = None
    while True:  # <1>
        term = yield average  # <2>
        total += term
        count += 1
        average = total/count
        
avg0 = averager0()
next(avg0)
print(avg0.send(10))
print(avg0.send(30))
print(avg0.send(5))

## with  decorator

In [ ]:
from functools import wraps

def coroutine(func):
    """Decorator: primes `func` by advancing to first `yield`"""
    @wraps(func)
    def primer(*args,**kwargs):
        gen = func(*args,**kwargs)
        next(gen)
        return gen
    return primer

@coroutine
def averager1():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count
        
avg1 = averager1()
print(avg1.send(10))
print(avg1.send(30))
print(avg1.send(5))

## with exception

In [ ]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:  # <2>
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')
    
demo = demo_exc_handling()
next(demo)

from inspect import getgeneratorstate

print(demo.send(11))
print(demo.send(22))

demo.throw(DemoException)
print(getgeneratorstate(demo))

try:
    demo.throw(ZeroDivisionError)
except:
    print('Error')
print(getgeneratorstate(demo))

# yield from

- delegating generator
- subgenerator
- caller

In [ ]:
def f1():
    yield range(5)

it1 = f1()

for i in it1:
    print(i)

In [ ]:
def f2():
    yield from range(5)

it2 = f2()
    
for i in it2:
    print(i)